**전에 ver1은 단일 클래스 분류에 컬러 마스크 인식 x**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger


Epoch 1/100
73/73 [==============================] - ETA: 0s - loss: 1.3112 - acc: 0.7903
Epoch 1: val_loss improved from inf to 1.81021, saving model to /content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


73/73 [==============================] - 187s 2s/step - loss: 1.3112 - acc: 0.7903 - val_loss: 1.8102 - val_acc: 0.8777 - lr: 1.0000e-04
Epoch 2/100
73/73 [==============================] - ETA: 0s - loss: 0.7857 - acc: 0.8925
Epoch 2: val_loss improved from 1.81021 to 1.15850, saving model to /content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5
73/73 [==============================] - 67s 910ms/step - loss: 0.7857 - acc: 0.8925 - val_loss: 1.1585 - val_acc: 0.8788 - lr: 1.0000e-04
Epoch 3/100
73/73 [==============================] - ETA: 0s - loss: 0.5980 - acc: 0.9110
Epoch 3: val_loss improved from 1.15850 to 0.85365, saving model to /content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5
73/73 [==============================] - 66s 904ms/step - loss: 0.5980 - acc: 0.9110 - val_loss: 0.8536 - val_acc: 0.8788 - lr: 1.0000e-04
Epoch 4/100
73/73 [==============================] - ETA: 0s - loss: 0.4723 - acc: 0.9300
Epoch 4

In [ ]:
# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# 고정된 시드 값
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# 하이퍼파라미터 설정
batch_size = 8
lr = 1e-4
epochs = 100
height = 384
width = 256
num_classes = 13

In [ ]:
# 데이터셋 경로
dataset_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug"

# 모델 및 로그 파일 경로
files_dir = os.path.join(r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug", "non-aug")
model_file = os.path.join(r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug", "unet-non-aug.h5")
log_file = os.path.join(r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug", "log-non-aug.csv")

In [ ]:
# 디렉터리 생성 함수
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
create_dir(files_dir)

In [ ]:
# Convolution Block
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [ ]:
# Encoder Block
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [ ]:
# Decoder Block
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    # Conv2DTranspose의 출력 크기 조정
    x = tf.image.resize(x, [tf.shape(skip)[1], tf.shape(skip)[2]], method='nearest')
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

In [ ]:
# U-Net Model
def build_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p4, 1024)

    # Decoder
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Output
    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)
    model = Model(inputs, outputs, name="UNET")
    return model

In [ ]:
# Data Loading and Preprocessing
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*.jpg")))
    train_y = sorted(glob(os.path.join(path, "train", "masks", "*.png")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.jpg")))
    test_y = sorted(glob(os.path.join(path, "test", "masks", "*.png")))
    return (train_x, train_y), (test_x, test_y)

In [ ]:
def read_image(path):
    # path.numpy().decode() 대신에 바로 경로를 사용
    x = cv2.imread(path.decode(), cv2.IMREAD_COLOR)
    x = cv2.resize(x, (width, height))
    x = x / 255.0
    x = x.astype(np.float32) # 데이터 타입을 float32로 변경
    return x

In [ ]:
def read_mask(path):
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_COLOR)
    mask = cv2.resize(mask, (width, height))
    mask = mask.astype(np.uint8)  # 정수형으로 변환

    # 클래스별 색상 정의
    class_colors = {
        (255, 0, 0): 0,  # S1
        (0, 255, 0): 1,  # L5
        (0, 0, 255): 2,  # L4
        (255, 255, 0): 3,  # L3
        (255, 0, 255): 4,  # L2
        (0, 255, 255): 5,  # L1
        (100, 100, 255): 6,  # T12
        (100, 255, 100): 7,  # T11
        (255, 100, 100): 8,  # T10
        (100, 255, 255): 9,  # T9
        (255, 100, 255): 10, # latLSpine
        (255, 255, 100): 11, # latSacrum
        (100, 100, 100): 12  # Aorticcalcification
    }

    # 색상을 클래스 인덱스로 변환
    class_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int32)
    for color, class_idx in class_colors.items():
        equality = np.equal(mask, color)
        class_mask[np.all(equality, axis=-1)] = class_idx

    class_mask = class_mask.astype(np.float32) #  여기서 데이터 타입을 float32로 설정
    class_mask = np.expand_dims(class_mask, axis=-1)  # 새로운 차원 추가
    return class_mask

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    return x, y

In [ ]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
# Data Load and Dataset Creation
(train_x, train_y), (test_x, test_y) = load_data(dataset_path)
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

In [ ]:
# Model Build and Compile
model = build_unet((height, width, 3))
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["acc"])

In [ ]:
# Callback Functions and Model Training
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [ ]:
# Model Training
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=callbacks
)